In [1]:
import os
import numpy as np
import open3d as o3d
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [25]:
SAMPLE_DATA_DIR = r"C:\Users\matha\Desktop\Source_codes\sereact\data\dl_challenge\9f50f3c0-9915-11ee-9103-bbb8eae05561"

In [26]:
def inspect_npy_file(file_path):
    data = np.load(file_path, allow_pickle=True)
    if isinstance(data,dict):
        print("Data is a dictionary with keys:")
        for key in data.keys():
            print(f" - {key}")

    print(f"Data type: {type(data)}")
    print(f"Data shape: {data.shape if hasattr(data, 'shape') else 'N/A'}")
    print(f"First element: {data[:1] if hasattr(data, '__getitem__') else 'N/A'}")
    return data

In [27]:
def inspect_pcd_file(file_path):
    data = np.load(file_path)
    x = data[0]
    y = data[1]
    z = data[2]

    print(f"X : Max - {x.max()} ::: Min = {x.min()}")
    print(f"Y : Max - {y.max()} ::: Min = {y.min()}")
    print(f"Z : Max - {z.max()} ::: Min = {z.min()}")


In [28]:
def visualize_point_cloud(pc: np.ndarray, mask: np.ndarray = None):
    if mask is not None:
        x = pc[0][mask]
        y = pc[1][mask]
        z = pc[2][mask]
    else:
        x = pc[0].flatten()
        y = pc[1].flatten()
        z = pc[2].flatten()

    points = np.vstack((x, y, z)).T

    # Remove invalid points
    points = points[~np.isnan(points).any(axis=1)]

    #Grey colour for all points
    if mask is not None:
        colors = np.zeros((points.shape[0], 3))  # Grey color
        colors[mask] = [0.5, 0.5, 0.5]  # Grey for masked points
    else:
        colors = np.full((points.shape[0], 3), [0.5, 0.5, 0.5])  # Grey for all points

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    o3d.visualization.draw_geometries([pcd])


In [29]:
def visualize_bboxes(bboxes: np.ndarray, pc: np.ndarray = None):
    vis_list = []
    x = pc[0].flatten()
    y = pc[1].flatten()
    z = pc[2].flatten()
    points = np.vstack((x, y, z)).T
    points = points[~np.isnan(points).any(axis=1)]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    vis_list.append(pcd)
    for box in bboxes:
        lines = [
            [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
            [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
            [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical lines
        ]

        colors = [[1, 0, 0] for _ in lines]  # Red color
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(box),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
        vis_list.append(line_set)

    o3d.visualization.draw_geometries(vis_list)


In [30]:
def visualize_scene(pc: np.ndarray, bboxes: np.ndarray, masks: np.ndarray):

    all_points = []
    all_colors = []

    for idx, mask in enumerate(masks):
        x = pc[0][mask]
        y = pc[1][mask]
        z = pc[2][mask]

        points = np.vstack((x, y, z)).T
        color = np.random.rand(3)  # Random color per instance
        colors = np.tile(color, (points.shape[0], 1))

        all_points.append(points)
        all_colors.append(colors)

    if not all_points:
        print("No valid points to visualize.")
        return

    all_points = np.concatenate(all_points, axis=0)
    all_colors = np.concatenate(all_colors, axis=0)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(all_points)
    pcd.colors = o3d.utility.Vector3dVector(all_colors)

    # Add bounding boxes
    vis_objects = [pcd]
    for box in bboxes:
        lines = [
            [0, 1], [1, 2], [2, 3], [3, 0],
            [4, 5], [5, 6], [6, 7], [7, 4],
            [0, 4], [1, 5], [2, 6], [3, 7]
        ]
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(box),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.paint_uniform_color([1, 0, 0])
        vis_objects.append(line_set)

    o3d.visualization.draw_geometries(vis_objects)


In [31]:
def visualize_2d_bboxes(image, bbox2d):
    if isinstance(image, Image.Image):
        image = np.array(image)

    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    for box in bbox2d:
        x_min, y_min, x_max, y_max = box
        width = x_max - x_min
        height = y_max - y_min
        rect = patches.Rectangle((x_min, y_min), width, height,
                                 linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    ax.set_title("2D Projected Bounding Boxes")
    plt.axis('off')
    plt.show()

In [32]:
for file in sorted(os.listdir(SAMPLE_DATA_DIR)):
    if file.endswith(".npy"):
        file_path = os.path.join(SAMPLE_DATA_DIR, file)
        print(f"Inspecting {file}...")
        data = inspect_npy_file(file_path)
        if file.split(".")[0] == "bbox3d":
            bbox3d = np.load(file_path, allow_pickle=True)
        elif file.split(".")[0] == "mask":
            mask = np.load(file_path, allow_pickle=True)
        elif file.split(".")[0] == "pc":
            pc = np.load(file_path, allow_pickle=True)
            inspect_pcd_file(file_path)
        elif file.split(".")[0] == "bbox2d":
            bbox2d = np.load(file_path, allow_pickle=True)
    elif file.endswith(".jpg"):
        file_path = os.path.join(SAMPLE_DATA_DIR, file)
        img = Image.open(file_path)
        print(f"Image size: {img.size}")
        
# Visualize the point cloud
visualize_point_cloud(pc)
# Visualize the bounding boxes
visualize_bboxes(bbox3d,pc)
# Visualize the scene with point cloud and bounding boxes
visualize_scene(pc, bbox3d, mask)
# visualize_2d_bboxes(img,bbox2d)

Inspecting bbox3d.npy...
Data type: <class 'numpy.ndarray'>
Data shape: (13, 8, 3)
First element: [[[ 0.18777966  0.03770728  0.9663663 ]
  [ 0.04846087  0.17649247  0.96368265]
  [ 0.05408309  0.18366596  1.0427881 ]
  [ 0.19340183  0.04488071  1.0454718 ]
  [ 0.07943203 -0.0707179   0.983899  ]
  [-0.05988673  0.06806732  0.98121536]
  [-0.0542646   0.07524075  1.0603209 ]
  [ 0.08505423 -0.06354444  1.0630045 ]]]
Inspecting mask.npy...
Data type: <class 'numpy.ndarray'>
Data shape: (13, 525, 765)
First element: [[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
Inspecting pc.npy...
Data type: <class 'numpy.ndarray'>
Data shape: (3, 525, 765)
First element: [[[-0.46917149 -0.46894402 -0.46499612 ...  0.48987612  0.48884636
    0.48970789]
  [-0.46998157 -0.46669856 -